# Assignment 2 - Data from the web

## Data scraping

## 1. Obtain all the data

In [32]:
import requests
from bs4 import BeautifulSoup

The first thing to do is to get the base URL (basically looking at the address bar of the browser).
We need a second variable INDEX_PARAM because this page is using frames, and we'll need the base address to access them.

In [33]:
BASE_URL = "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm"
INDEX_PARAM = "ww_i_reportmodel=133685247"

Then let's fetch the parameters names, but as it is a bit tricker, BeautifulSoup will be a great help for this (and also some manual html inspection).

In [34]:
r = requests.get(BASE_URL + '?' + INDEX_PARAM)
soup = BeautifulSoup(r.text, 'html.parser')

for frame in soup.find_all('frame'):
    print(frame['name'])
    #unfinished

entete
toc
principal


and finally a nice function to easily query the EPFL page and get the data from it.

In [35]:
def query_epfl( args ):
    "Query the base URL with passed arguments (args)"
    print(args)
    return